## CellProfiler-OMERO demo
We have setup an example notebook to guide you through the steps to analise images stored in OMERO using CellProfiler

Let's start by importing some libraries we are going to need

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import ezomero

# Make CellProfiler run without a GUI
import cellprofiler_core.preferences as cp_preferences
cp_preferences.set_headless()

from cp_omero_toolbox import *

import tempfile

In [ ]:
# Tell CellProfiler to get input from and save output in a temp directory
output_dir = tempfile.TemporaryDirectory()
input_dir = tempfile.TemporaryDirectory()
cp_preferences.set_default_output_directory(output_dir.name)
cp_preferences.set_default_image_directory(input_dir.name)

Let's connect to OMERO. When we connect we get a connection object that we will have to use in every interaction with OMERO.

In [ ]:
# Creating a connection object
host = "bioimage.france-bioinformatique.fr"
port = 4074
conn = ezomero.connect(host=host, port=port)

# Connecting
conn.connect()
# The connection will timeout after a period of inactivity. To avoid that we can tell our new connection to say "Hi, I'm still here"
conn.c.enableKeepAlive(60)
# Let's verify that we are connected
conn.isConnected()

Time to grasp a Dataset from OMERO and download a CellProfiler pipeline that is attached to it. Go to the browser, select a dataset and copy the ID.

In [ ]:
dataset_id = int(input("Dataset id: "))


We create a new pipeline with that file, and we remove the first 4 modules. The first 4 modules are in charge of preparing the image data when they are loaded from disk. We don't need them here because we are using OMERO.


In [ ]:
measurements_dfs = run_cp_pipeline(conn=conn,
                                   dataset_id=dataset_id,
                                   objects_to_image_table="spots",
                                   objects_to_mask=["nuclei"],
                                   objects_to_point=["spots"],
                                   output_dir=output_dir,
                                   input_dir=input_dir)

We can now start feeding images into the pipeline

In [ ]:
cols = measurements_dfs["Image"].columns
vals = [measurements_dfs["Image"][c].values.tolist() for c in cols]

print(len(vals))
print(vals)



In [ ]:
# remove the output directory
output_dir.cleanup()

# and close the connection to the OMERO server
conn.close()